In [ ]:
import pandas as pd
from pathlib import Path
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely import wkt
from functools import reduce


## Merge Barcelona info into time series

In [ ]:
def merge_csv_per_year(name, yearStart, yearEnd, input_dir, output_path=None, verbose=True):
    """
    Merge yearly files named {year}_{name}.csv for years 2015-2025
    into a single CSV named yearStart-yearEnd_{name}.csv by default.

    Parameters:
    - input_dir: path to the folder containing the yearly CSVs (string or Path)
    - output_path: optional path (string or Path) for the merged CSV file
    - verbose: print progress/messages if True

    Returns:
    - pandas.DataFrame with the concatenated data
    """
    input_dir = Path(input_dir)
    years = range(yearStart, yearEnd + 1)
    dfs = []
    for y in years:
        fp = input_dir / f"{y}_{name}.csv"
        if fp.exists():
            try:
                df = pd.read_csv(fp)
                df = df.iloc[:, 1:]     # Drop first column since it's a date 
                df['year'] = y
                dfs.append(df)
                if verbose:
                    print(f"Loaded: {fp.name} ({len(df)} rows)")
            except Exception as e:
                if verbose:
                    print(f"Failed to read {fp}: {e}")
        else:
            if verbose:
                print(f"Missing: {fp.name}")
    if not dfs:
        raise FileNotFoundError("No input files found for 2015-2025 in " + str(input_dir))
    out_df = pd.concat(dfs, ignore_index=True, sort=False)
    out_path = Path(output_path) if output_path else input_dir / f"{yearStart}-{yearEnd}_{name}.csv"
    out_df.to_csv(out_path, index=False)
    if verbose:
        print(f"Saved merged CSV to: {out_path} ({len(out_df)} total rows)")
    return out_df



In [ ]:
merge_csv_per_year(name="loc_hab_valors", yearStart=2018, yearEnd=2025, input_dir="../data/raw/barcelona", output_path="../data/preprocessed/barcelona/2018-2025_loc_hab_valors.csv", verbose=True)

In [ ]:
merge_csv_per_year(name="pad_mdbas_niv-educa-esta_sexe", yearStart=2015, yearEnd=2025, input_dir="../data/raw/barcelona", output_path="../data/preprocessed/barcelona/2018-2025_pad_mdbas_niv-educa-esta_sexe.csv", verbose=True)

In [ ]:
merge_csv_per_year(name="atles_renda_bruta_llar", yearStart=2015, yearEnd=2023, input_dir="../data/raw/barcelona", output_path="../data/preprocessed/barcelona/2018-2025_atles_renda_bruta_llar.csv", verbose=True)

In [ ]:
merge_csv_per_year(name="pad_mdba_sexe_edat-1", yearStart=2018, yearEnd=2025, input_dir="../data/raw/barcelona", output_path="../data/preprocessed/barcelona/2018-2025-pad_mdba_sexe_edat-1.csv", verbose=True)

# Export Barcelona geometric data to GeoJSON

In [ ]:

gdf = gpd.read_file("../data/raw/barcelona/BCN_UNITATS_ADM/0301040100_Barris_UNITATS_ADM.shp")
gdf = gdf.to_crs(epsg=4326) # epsg 4326 = WGS84 standard for lat/lon
# Save to GeoJSON
gdf.to_file("../data/preprocessed/barcelona/barcelona_neighborhoods.geojson", driver="GeoJSON")

# Load and Inspect Preprocessed Data

In [ ]:
# Load the datasets
try:
    property_values_df = pd.read_csv('../data/preprocessed/barcelona/2018-2025_loc_hab_valors.csv')
    income_df = pd.read_csv('../data/preprocessed/barcelona/2018-2025_atles_renda_bruta_llar.csv')
    studies_df = pd.read_csv('../data/preprocessed/barcelona/2018-2025_pad_mdbas_niv-educa-esta_sexe.csv')
    age_df = pd.read_csv('../data/preprocessed/barcelona/2018-2025-pad_mdba_sexe_edat-1.csv')
    neighborhoods_gdf = gpd.read_file('../data/preprocessed/barcelona/barcelona_neighborhoods.geojson')
except FileNotFoundError as e:
    print(f"Error loading data: {e}. Please check your file paths.")
    exit(1)

# Initial Inspection
print("Property Values Info:")
property_values_df.info()
print("\nIncome Info:")
income_df.info()
print("\nStudies Info:")
studies_df.info()
print("\nAge and Population Distribution Info:")
age_df.info()
print("\nNeighborhoods GeoDataFrame Info:")
neighborhoods_gdf.info()

# Display first few rows to understand the content
print("\nProperty Values Head:")
print(property_values_df.head())
print("\nIncome Head:")
print(income_df.head())
print("\nStudies Head:")
print(studies_df.head())
print("\nAge and Population Distribution Head:")
print(age_df.head())
print("\nNeighborhoods Head:")
print(neighborhoods_gdf.head())

# print the number of unique neighborhoods in each dataset
print(f"\nUnique neighborhoods in Property Values: {property_values_df['Nom_barri'].nunique()}")




In [ ]:
# show unique values in the column "Desc_valors" of property_values_df
print(property_values_df['Desc_valors'].unique())

# Standardize Column Names & Clean 'Valor' Columns

In [ ]:
def standardize_and_clean(df, valor_col='Valor'):
    # Standardize neighborhood and district codes/names
    df.rename(columns={
        'Codi_Districte': 'Codi_districte', 'Nom_Districte': 'Nom_districte',
        'Codi_Barri': 'Codi_barri', 'Nom_Barri': 'Nom_barri'
    }, inplace=True)
    # Convert 'Valor' from object to numeric, treating errors as NaN and filling with 0
    if valor_col in df.columns and df[valor_col].dtype == 'object':
        df[valor_col] = pd.to_numeric(df[valor_col], errors='coerce').fillna(0)
    return df

property_values_df = standardize_and_clean(property_values_df, valor_col='Valors')
income_df = standardize_and_clean(income_df, valor_col='Import_Renda_Bruta_€')
studies_df = standardize_and_clean(studies_df)
age_df = standardize_and_clean(age_df)




# Process Each Feature According to the Data Model


In [ ]:

# A. Median Property Price (from total cadastral value)
prop_price_total = property_values_df[property_values_df['Desc_valors'] == 'Valor_cadastral_total_€']
median_property_price = prop_price_total.groupby(['Codi_barri', 'year'])['Valors'].median().reset_index()
median_property_price.rename(columns={'Valors': 'median_property_price'}, inplace=True)

# B. Average Price per m2 (from unitary cadastral value)
prop_price_m2 = property_values_df[property_values_df['Desc_valors'] == 'Valor_cadastral_unitari_€/m2']
avg_price_per_m2 = prop_price_m2.groupby(['Codi_barri', 'year'])['Valors'].median().reset_index()
avg_price_per_m2.rename(columns={'Valors': 'avg_price_per_m2'}, inplace=True)

# C. Median Household Income
median_household_income = income_df.groupby(['Codi_barri', 'year'])['Import_Renda_Bruta_€'].median().reset_index()
median_household_income.rename(columns={'Import_Renda_Bruta_€': 'median_household_income'}, inplace=True)

# D. Percentage Higher Education
# First, get total population from the studies dataset
total_pop_studies = studies_df.groupby(['Codi_barri', 'year'])['Valor'].sum().reset_index()
total_pop_studies.rename(columns={'Valor': 'total_population_studies'}, inplace=True)
# Then, get population with higher education (levels 5 and 6)
higher_edu_pop = studies_df[studies_df['NIV_EDUCA_esta'].isin([5, 6])]
higher_edu_counts = higher_edu_pop.groupby(['Codi_barri', 'year'])['Valor'].sum().reset_index()
higher_edu_counts.rename(columns={'Valor': 'population_higher_education'}, inplace=True)
# Merge and calculate percentage
education_features = pd.merge(total_pop_studies, higher_edu_counts, on=['Codi_barri', 'year'])
education_features['pct_higher_education'] = (education_features['population_higher_education'] / education_features['total_population_studies']) * 100

# E. Population Density and Percentage of Young Adults (from the Age dataset)
# Get total population
total_population = age_df.groupby(['Codi_barri', 'year'])['Valor'].sum().reset_index()
total_population.rename(columns={'Valor': 'total_population'}, inplace=True)
# Get young adult population (ages 20-39 inclusive)
young_adults_pop = age_df[age_df['EDAT_1'].between(20, 39)]
young_adult_counts = young_adults_pop.groupby(['Codi_barri', 'year'])['Valor'].sum().reset_index()
young_adult_counts.rename(columns={'Valor': 'young_adult_population'}, inplace=True)
# Merge population features together
population_features = pd.merge(total_population, young_adult_counts, on=['Codi_barri', 'year'])
# Calculate pct_young_adults
population_features['pct_young_adults'] = (population_features['young_adult_population'] / population_features['total_population']) * 100



# Handle Geospatial Data (for population density)


In [ ]:
# Load the CORRECT shapefile directly
neighborhoods_gdf = gpd.read_file("../data/raw/barcelona/BCN_UNITATS_ADM/0301040100_Barris_UNITATS_ADM.shp")

# --- Simplified Geospatial Processing ---

# STEP 1: Rename the 'BARRI' column to 'Codi_barri' for consistency
neighborhoods_gdf.rename(columns={'BARRI': 'Codi_barri'}, inplace=True)

# STEP 2: Robustly convert the neighborhood code to integer
neighborhoods_gdf['Codi_barri'] = pd.to_numeric(neighborhoods_gdf['Codi_barri'], errors='coerce')
neighborhoods_gdf.dropna(subset=['Codi_barri'], inplace=True)
neighborhoods_gdf['Codi_barri'] = neighborhoods_gdf['Codi_barri'].astype(int)

# STEP 3: Proceed with the area calculation
neighborhoods_gdf_proj = neighborhoods_gdf.to_crs(epsg=25831)
neighborhoods_gdf['area_km2'] = neighborhoods_gdf_proj.geometry.area / 1_000_000

# STEP 4: Merge with population data
# We use the cleaned 'neighborhoods_gdf' which is now our main geo-dataframe
population_features = pd.merge(population_features, neighborhoods_gdf[['Codi_barri', 'area_km2']], on='Codi_barri')
population_features['population_density'] = population_features['total_population'] / population_features['area_km2']

## Mix everything

In [ ]:
feature_dfs = [
    median_household_income,
    avg_price_per_m2,
    education_features[['Codi_barri', 'year', 'pct_higher_education']],
    population_features[['Codi_barri', 'year', 'population_density', 'pct_young_adults']]
]

# Assemble the Master DataFrame 


In [ ]:
# (Code to create master_df from feature_dfs remains the same)
master_df = reduce(lambda left, right: pd.merge(left, right, on=['Codi_barri', 'year'], how='inner'), feature_dfs)

# Add identifiers and geometry
master_df['city'] = 'Barcelona'
 
# Merge geometry and neighborhood names
master_df = pd.merge(master_df, neighborhoods_gdf[['Codi_barri', 'NOM', 'geometry']], on='Codi_barri', how='left')

# NOTE: I also changed 'NDESCR_CA' to 'NOM' based on typical Shapefile column names for the neighborhood name.
# Check your neighborhoods_gdf.columns to be sure, but 'NOM' is very likely the correct name column.

master_df.rename(columns={'NOM': 'neighborhood_name', 'Codi_barri': 'neighborhood_id'}, inplace=True)
master_df['neighborhood_id'] = 'BCN_' + master_df['neighborhood_id'].astype(str)

# Display the result
print("--- Master DataFrame Info ---")
master_df.info()
print("\n--- Master DataFrame Head ---")
print(master_df.head())

# Save the master DataFrame to a csv file
master_df.to_csv("../data/preprocessed/barcelona/barcelona_master_dataframe.csv", index=False)
